In [26]:
import pandas as pd
twitter = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = 'latin1', header=None)
twitter = twitter.sample(n = 75000)

In [27]:
import re
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t

def labelFilter(label):
  if label == 4:
    return 1
  return label

twitter=twitter[[0,5]]
twitter[5]=twitter[5].apply(preprocess)
twitter[0] = twitter[0].apply(labelFilter)
twitter

,0,5
998868,1,good luck. I hope you found some companies on...
1098527,1,new haircut agaiin woohooo its sooo warm time...
41787,0,just wants to get some sleep...but I cant caus...
415282,0,Watching a scary movie
1501095,1,i was in sigma delta tau
...,...,...
676555,0,Yeah. Ima delete it and redownload when I get...
1581392,1,Is waiting for Marcel and Choles choice of Wed...
351580,0,wish I knew as much as I thought I did
266613,0,i miss you sister! &lt;3 i eat this young coc...


In [3]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import tensorflow_text as text

  Using cached tensorflow_text-2.7.3-cp38-cp38-win_amd64.whl (2.5 MB)


In [4]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

sentences = ["I love you", "I hate you"]
preprocessed_text = bert_preprocess(sentences)
bert_encoder(preprocessed_text)["pooled_output"]

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-9.1058302e-01, -3.5070494e-01, -3.2318407e-01, ...,
        -4.5425259e-05, -7.1246165e-01,  9.1118795e-01],
       [-8.1203628e-01, -2.2017553e-01,  3.5842058e-01, ...,
         4.2030022e-01, -5.8162069e-01,  8.2115519e-01]], dtype=float32)>

In [28]:
data, labels = twitter[5], twitter[0]

example_data, example_labels = data[:50], labels[:50]

In [8]:
text_input = keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

dense = keras.layers.Dense(1, activation='sigmoid')(outputs["pooled_output"])

model = keras.Model(inputs=[text_input], outputs=[dense])
model.summary()
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(example_data, example_labels, epochs=2)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['input_2[0][0]']                
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [10]:
test = pd.read_csv("testdata.manual.2009.06.14.csv", encoding="latin1", header=None)

In [16]:
test = test[[0,5]]
test[5]=test[5].apply(preprocess)
test[0] = test[0].apply(labelFilter)
test = test[test[0] != 2]

test_data, test_labels = test[5], test[0]

In [ ]:
import numpy as np
y_pred = np.where(model.predict(test_data) > .5, 1, 0)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, y_pred))


              precision    recall  f1-score   support

           0       0.49      1.00      0.66       177
           1       0.00      0.00      0.00       182

    accuracy                           0.49       359
   macro avg       0.25      0.50      0.33       359
weighted avg       0.24      0.49      0.33       359



C:\Users\Nick\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nick\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nick\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
